The purpose of this project is to generate evidence using Evidence Model.

---

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [12]:
import os

listOfFiles = os.listdir('/content/gdrive/MyDrive/tdqfs/documentsForGeneratingQuery')

content = []
content_dir = []

for dir in listOfFiles:
  text = ''
  with open('/content/gdrive/MyDrive/tdqfs/documentsForGeneratingQuery/'+str(dir), "r") as d:
    for doc in d:
      text += doc
      text += ' '
    content.append(text)
    content_dir.append(dir)

print(len(content))

40


In [14]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
!pip install transformers
!pip install sentencepiece==0.1.91

In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/MyDrive/EvidenceQuery/model').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def generateEvidence (text_document):
  input_ids = tokenizer.encode(text_document, max_length=512, padding=True, truncation = True, return_tensors='pt').to(device)
  output = model.generate(input_ids, max_length=7, min_length=7, no_repeat_ngram_size= 4)
  evidence = tokenizer.decode(output[0], skip_special_tokens=True)
  return evidence

#print(summary[200])
#print(generateEvidence(content[200]))

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:169: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [17]:
from tqdm import tqdm
evidence = []

for i in tqdm(range(len(content))):
   evidence.append(generateEvidence(content[i]))

100%|██████████| 40/40 [00:05<00:00,  7.52it/s]


In [18]:
import json

tdqfs_data = []

for i in range(len(content)):
  tdqfs_data.append({content_dir[i]:evidence[i]})

with open('/content/gdrive/MyDrive/EvidenceQuery/tdqfs_query.json', 'w') as fp:
  json.dump(tdqfs_data, fp)